In [1]:
import numpy as np
import pandas as pd

In [2]:
books=pd.read_csv('Books.csv')
ratings=pd.read_csv('Ratings.csv')
users=pd.read_csv('Users.csv')

C:\Users\Nikhi\AppData\Local\Temp\ipykernel_23384\775865190.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('Books.csv')


In [3]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [4]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [5]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [6]:
print(books.duplicated().sum(), ratings.duplicated().sum(), users.duplicated().sum())

0 0 0


Popularity Filtering

In [8]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [9]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [22]:
df=books.merge(ratings,on='ISBN')[['Book-Title','Book-Rating']]
df.head()

,Book-Title,Book-Rating
0,Classical Mythology,0
1,Clara Callan,5
2,Clara Callan,0
3,Clara Callan,8
4,Clara Callan,0


In [27]:
rating_num=df.groupby('Book-Title').count()['Book-Rating'].reset_index()
rating_num.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
rating_num.head()

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [28]:
rating_avg=df.groupby('Book-Title').mean()['Book-Rating'].reset_index()
rating_avg.rename(columns={'Book-Rating':'avg_ratings'},inplace=True)
rating_avg.head()

,Book-Title,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [30]:
popular=rating_num.merge(rating_avg,on='Book-Title')
popular.head()

,Book-Title,num_ratings,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.25
1,Always Have Popsicles,1,0.00
2,Apple Magic (The Collector's series),1,0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.00
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.00


In [31]:
popular.describe()

,num_ratings,avg_ratings
count,241071.000000,241071.000000
mean,4.277312,2.890137
std,16.738685,3.268984
min,1.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.971429
75%,3.000000,5.000000
max,2502.000000,10.000000


In [41]:
#fetching those books which were rated by more than 250 users=popular books

popular=popular[popular['num_ratings']>250].sort_values(by='avg_ratings',ascending=False)
popular

,Book-Title,num_ratings,avg_ratings
80434,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
80422,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
80441,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
80426,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
80414,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
...,...,...,...
227701,Vinegar Hill (Oprah's Book Club (Paperback)),265,2.245283
233635,Whispers,286,2.199301
143377,Presumed Innocent,294,2.139456
94382,Isle of Dogs,288,2.000000


In [ ]:
popular=popular.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','num_ratings','avg_ratings','Book-Author','Image-URL-M']]

In [45]:
popular

,Book-Title,num_ratings,avg_ratings,Book-Author,Image-URL-M
0,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804,J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...
3,Harry Potter and the Goblet of Fire (Book 4),387,5.824289,J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...
5,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410,J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...
9,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...
13,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453,J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...
...,...,...,...,...,...
716,Vinegar Hill (Oprah's Book Club (Paperback)),265,2.245283,A. Manette Ansay,http://images.amazon.com/images/P/0380730138.0...
717,Whispers,286,2.199301,BELVA PLAIN,http://images.amazon.com/images/P/0440216745.0...
727,Presumed Innocent,294,2.139456,Scott Turow,http://images.amazon.com/images/P/0446359866.0...
733,Isle of Dogs,288,2.000000,Patricia Cornwell,http://images.amazon.com/images/P/0425182908.0...


In [140]:
popular['avg_ratings']=round(popular['avg_ratings'],ndigits=1)

In [141]:
popular.head()

,Book-Title,num_ratings,avg_ratings,Book-Author,Image-URL-M
0,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.8,J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...
3,Harry Potter and the Goblet of Fire (Book 4),387,5.8,J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...
5,Harry Potter and the Sorcerer's Stone (Book 1),278,5.7,J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...
9,Harry Potter and the Order of the Phoenix (Boo...,347,5.5,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...
13,Harry Potter and the Chamber of Secrets (Book 2),556,5.2,J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...


In [143]:
popular.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 0 to 739
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Book-Title   186 non-null    object 
 1   num_ratings  186 non-null    int64  
 2   avg_ratings  186 non-null    float64
 3   Book-Author  186 non-null    object 
 4   Image-URL-M  186 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 8.7+ KB


Collaborative Filtering

In [ ]:
#users who rated >= 250, books rated more than 50+ users

In [48]:
d=books.merge(ratings,on='ISBN')[['User-ID','Book-Title','Book-Rating',]]
d

,User-ID,Book-Title,Book-Rating
0,2,Classical Mythology,0
1,8,Clara Callan,5
2,11400,Clara Callan,0
3,11676,Clara Callan,8
4,41385,Clara Callan,0
...,...,...,...
1031131,276463,There's a Bat in Bunk Five,7
1031132,276579,From One to One Hundred,4
1031133,276680,Lily Dale : The True Story of the Town that Ta...,0
1031134,276680,Republic (World's Classics),0


In [57]:
y=d.groupby('User-ID').count()['Book-Rating']>250
experienced_readers=y[y].index

In [66]:
filtered_users=d[d['User-ID'].isin(experienced_readers)]
filtered_users.head()

,User-ID,Book-Title,Book-Rating
3,11676,Clara Callan,8
6,85526,Clara Callan,0
10,177458,Clara Callan,0
21,110912,Flu: The Story of the Great Influenza Pandemic...,10
26,197659,Flu: The Story of the Great Influenza Pandemic...,9


In [71]:
x=filtered_users.groupby('Book-Title').count()['Book-Rating']>=50
liked_books=x[x].index

In [72]:
filtered_users=filtered_users[filtered_users['Book-Title'].isin(liked_books)]
filtered_users.head()

,User-ID,Book-Title,Book-Rating
31,11676,The Kitchen God's Wife,9
33,36836,The Kitchen God's Wife,0
34,46398,The Kitchen God's Wife,9
38,113270,The Kitchen God's Wife,0
39,113519,The Kitchen God's Wife,0


In [75]:
piv=filtered_users.pivot_table(columns='User-ID',index='Book-Title',values='Book-Rating')

In [77]:
piv.fillna(0,inplace=True)

In [80]:
piv

User-ID,254,2276,2766,3363,4385,6251,6543,6575,7158,7346,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(piv)

In [87]:
np.where(piv.index=='1984')[0][0]

0

In [102]:
sorted(list(enumerate(similarity[9])),key=lambda x: x[1])[1:6]

[(5, 0.0), (8, 0.0), (11, 0.0), (14, 0.0), (16, 0.0)]

In [134]:
def recommend(book):
    
    movie_index=np.where(piv.index==book)[0][0]
    distance=sorted(list(enumerate(similarity[movie_index])),key=lambda x: x[1],reverse=True)[1:5]
    
    
    data=[]
    for i in distance:
        #print(piv.index[i[0]])
        
        items=[]
        temp_df=books[books['Book-Title']==piv.index[i[0]]]
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(items)
        
    return data


In [133]:
recommend('1984')

[["The Handmaid's Tale",
  'Margaret Atwood',
  'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg'],
 ['The Hours : A Novel',
  'Michael Cunningham',
  'http://images.amazon.com/images/P/0312243022.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg']]

In [146]:
piv.iloc[0:10,]

User-ID,254,2276,2766,3363,4385,6251,6543,6575,7158,7346,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Case of Need,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
"A Child Called \It\"": One Child's Courage to Survive""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Civil Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Fine Balance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
import pickle

pickle.dump(popular,open('popular1.pkl','wb'))
pickle.dump(similarity,open('similarity_scores.pkl','wb'))
pickle.dump(piv,open('pivot_table.pkl','wb'))
pickle.dump(books,open('book.pkl','wb'))